In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
btc_data = pd.read_csv('../soc-sign-bitcoinotc.csv', header=None)
btc_data

,0,1,2,3
0,6,2,4,1.289242e+09
1,6,5,2,1.289242e+09
2,1,15,1,1.289243e+09
3,4,3,7,1.289245e+09
4,13,16,8,1.289254e+09
5,13,10,8,1.289254e+09
6,7,5,1,1.289363e+09
7,2,21,5,1.289371e+09
8,2,20,5,1.289371e+09
9,21,2,5,1.289381e+09


In [3]:
min(min(btc_data[1]), min(btc_data[0]))

1

In [3]:
grpedbyoutgoing = btc_data.groupby(0).count()
grpedbyincoming = btc_data.groupby(1).count()
print("Average outgoing degree - ", grpedbyoutgoing[1].mean())
print("Average incoming degree - ", grpedbyincoming[0].mean())

Average outgoing degree -  7.393435812214375
Average incoming degree -  6.075793786275179


In [4]:
btc_data.groupby([0,1]).count()[2]
#implies no multiple edges between any pair of nodes

0     1   
1     2       1
      3       1
      4       1
      5       1
      6       1
             ..
5994  5458    1
5995  35      1
5996  5949    1
5999  3878    1
6000  6002    1
Name: 2, Length: 35592, dtype: int64

In [5]:
grpbyrating = btc_data.groupby(2).count()
grpbyrating

,0,1,3
2,,,
-10,2413,2413,2413
-9,20,20,20
-8,31,31,31
-7,14,14,14
-6,5,5,5
-5,179,179,179
-4,27,27,27
-3,91,91,91
-2,182,182,182


In [5]:
#dividing the data into 137 timestamps according to paper evolve gcn
mintstep = btc_data[3].min()
maxtstep = btc_data[3].max()
print(mintstep, maxtstep)

1289241911.72836 1453684323.75728


In [6]:
pts = np.linspace(mintstep ,maxtstep, 138)
intervals = []
for i in range(len(pts)):
    if i!= len(pts) - 1:
        intervals.append((pts[i], pts[i+1]))
timestamp_edges = list(map(lambda inte:  btc_data[(btc_data[3] >= inte[0]) & (btc_data[3] < inte[1]) ], intervals))

In [7]:
#Divide according to evolvegcn
train_timest = timestamp_edges[:95]
val_timest = timestamp_edges[95:109]
test_timest = timestamp_edges[109:]
print(len(train_timest), len(val_timest), len(test_timest))

95 14 28


In [8]:
print("Training- Avg number of edges and nodes")
print(np.mean(list(map(lambda gr: gr.shape[0], train_timest))))
print(np.mean(list(map(lambda gr: len(set(gr[0]).union(gr[1])), train_timest))))
print("Validation- Avg number of edges and nodes")
print(np.mean(list(map(lambda gr: gr.shape[0], val_timest))))
print(np.mean(list(map(lambda gr: len(set(gr[0]).union(gr[1])), val_timest))))
print("Test- Avg number of edges and nodes")
print(np.mean(list(map(lambda gr: gr.shape[0], test_timest))))
print(np.mean(list(map(lambda gr: len(set(gr[0]).union(gr[1])), test_timest))))

Training- Avg number of edges and nodes
344.89473684210526
187.78947368421052
Validation- Avg number of edges and nodes
126.71428571428571
100.28571428571429
Test- Avg number of edges and nodes
37.57142857142857
35.42857142857143


In [45]:
with open('btcotc_train.pickle', 'wb') as handle:
    pickle.dump(train_timest, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('btcotc_val.pickle', 'wb') as handle:
    pickle.dump(val_timest, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('btcotc_test.pickle', 'wb') as handle:
    pickle.dump(test_timest, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [11]:
train_timest[-2]

,0,1,2,3
32451,5596,4197,1,1.400871e+09
32452,5155,5353,2,1.400873e+09
32453,5472,2110,3,1.400873e+09
32454,3452,3909,1,1.400874e+09
32455,5353,5155,2,1.400875e+09
32456,5472,5596,1,1.400876e+09
32457,5596,5472,1,1.400876e+09
32458,5472,5597,1,1.400887e+09
32459,5597,5472,1,1.400888e+09
32460,2110,5472,3,1.400888e+09


In [18]:
trainratings[1]

,0,1,3
2,,,
1,18,18,18
2,4,4,4
3,5,5,5
5,1,1,1
8,1,1,1
9,2,2,2
